<a href="https://colab.research.google.com/github/rawbil/models_V1/blob/main/titanic_model/csvs/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict Survival in the Titanic

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore') # Ignore warnings and focus on main output